In [20]:
# セル1: 必要なライブラリのインポート
import json
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union


In [21]:
# セル2: JSONファイルの読み込み

# JSONファイルのパスを指定
file_path = '../rplan_json/shifted_external/1014_processed.json'  # 必要に応じてファイルパスを変更

# ファイルからJSONデータを読み込み
with open(file_path, 'r') as f:
    data = json.load(f)

# 必要なデータを取得
rooms_data = data['processed_data']['stage5_final_alignment']['rooms']
doors_data = data['processed_data']['stage5_final_alignment']['doors']
external_walls_data = data['processed_data']['stage5_final_alignment']['external_walls']


#### グリッドポイント（ノード）の生成

In [22]:
# セル3: グリッドポイントの生成

grid_unit = 5
max_coordinate = 225

# x座標とy座標の範囲を設定
x_coords = np.arange(0, max_coordinate + grid_unit, grid_unit)
y_coords = np.arange(0, max_coordinate + grid_unit, grid_unit)

# ノードIDの初期化
node_id = 0

# ノードのリストを格納するリスト
nodes_list = []

# グリッドポイントの生成
for x in x_coords:
    for y in y_coords:
        nodes_list.append({
            'id': node_id,
            'x': x,
            'y': y,
            'on_external_wall': 0,
            'on_internal_wall': 0,
            'outside_boundary': 0,
            'inside_boundary': 0
        })
        node_id += 1

# ノードの総数を表示
print(f'総ノード数: {len(nodes_list)}')


総ノード数: 2116


#### 部屋のポリゴンの作成

In [23]:
# セル4: 部屋のポリゴンの作成

room_polygons = []

for room in rooms_data:
    coordinates = room['polygon']['coordinates'][0]
    polygon = Polygon(coordinates)
    room_polygons.append(polygon)


#### フロアプラン全体の境界の作成
全ての部屋ポリゴンを統合して、フロアプランの境界を作成します。

In [24]:
# セル5: フロアプランの境界の作成

# 部屋ポリゴンのユニオンを計算
floor_plan_boundary = unary_union(room_polygons)


In [25]:
# セル6: 外壁のラインの作成

external_wall_lines = []

for wall in external_walls_data:
    p1 = wall['p1']
    p2 = wall['p2']
    line = LineString([p1, p2])
    external_wall_lines.append(line)


In [26]:
external_wall_lines = []
for wall in external_walls_data:
    p1 = wall['p1']
    p2 = wall['p2']
    line = LineString([p1, p2])
    external_wall_lines.append(line)

# Cell 7: Create door lines
door_lines = []
for door in doors_data:
    p1 = door['p1']
    p2 = door['p2']
    line = LineString([p1, p2])
    door_lines.append(line)

In [27]:
# Internal walls are edges from room polygons that are not external walls or doors
internal_wall_lines = []

# First, get all edges from room polygons
all_wall_lines = []
for polygon in room_polygons:
    coords = list(polygon.exterior.coords)
    num_coords = len(coords)
    for i in range(num_coords - 1):
        p1 = coords[i]
        p2 = coords[i + 1]
        line = LineString([p1, p2])
        all_wall_lines.append(line)

# Remove duplicate lines (considering reversed lines as duplicates)
unique_wall_lines = []
for line in all_wall_lines:
    is_duplicate = False
    for existing_line in unique_wall_lines:
        if line.equals(existing_line) or line.equals(LineString(list(existing_line.coords)[::-1])):
            is_duplicate = True
            break
    if not is_duplicate:
        unique_wall_lines.append(line)

# Identify internal wall lines
for line in unique_wall_lines:
    is_external = any(line.equals(ext_line) for ext_line in external_wall_lines)
    is_door = any(line.equals(door_line) for door_line in door_lines)
    if not is_external and not is_door:
        internal_wall_lines.append(line)

In [28]:

# セル8: ノードの特徴量の計算

from shapely.geometry import Point

for node in nodes_list:
    point = Point(node['x'], node['y'])
    
    # Determine if inside or outside boundary
    if floor_plan_boundary.contains(point) or floor_plan_boundary.touches(point):
        node['inside_boundary'] = 1
        node['outside_boundary'] = 0
    else:
        node['inside_boundary'] = 0
        node['outside_boundary'] = 1
    
    # Initialize wall flags
    node['on_external_wall'] = 0
    node['on_internal_wall'] = 0
    
    # Check if on external wall
    for line in external_wall_lines:
        if line.distance(point) < 1e-6:
            node['on_external_wall'] = 1
            break  # Cannot be on both external and internal wall
    
    # Check if on internal wall, only if not on external wall
    if node['on_external_wall'] == 0:
        for line in internal_wall_lines:
            if line.distance(point) < 1e-6:
                node['on_internal_wall'] = 1
                break


In [29]:
# セル9: ノードデータのデータフレーム化

nodes_df = pd.DataFrame(nodes_list)

# データフレームの先頭を表示
nodes_df.head()


,id,x,y,on_external_wall,on_internal_wall,outside_boundary,inside_boundary
0,0,0,0,1,0,0,1
1,1,0,5,1,0,0,1
2,2,0,10,1,0,0,1
3,3,0,15,1,0,0,1
4,4,0,20,1,0,0,1


In [30]:
# セル10: 特徴量の統計情報を表示

nodes_df.describe()


,id,x,y,on_external_wall,on_internal_wall,outside_boundary,inside_boundary
count,2116.000000,2116.000000,2116.000000,2116.000000,2116.000000,2116.000000,2116.000000
mean,1057.500000,112.500000,112.500000,0.047259,0.021739,0.731096,0.268904
std,610.980905,66.395281,66.395281,0.212242,0.145865,0.443495,0.443495
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,528.750000,55.000000,55.000000,0.000000,0.000000,0.000000,0.000000
50%,1057.500000,112.500000,112.500000,0.000000,0.000000,1.000000,0.000000
75%,1586.250000,170.000000,170.000000,0.000000,0.000000,1.000000,1.000000
max,2115.000000,225.000000,225.000000,1.000000,1.000000,1.000000,1.000000


In [31]:
# セル11: 特定の条件を持つノードの数

num_on_external_wall = nodes_df['on_external_wall'].sum()
num_on_internal_wall = nodes_df['on_internal_wall'].sum()
num_inside_boundary = nodes_df['inside_boundary'].sum()
num_outside_boundary = nodes_df['outside_boundary'].sum()

print(f'Number of nodes on external wall: {num_on_external_wall}')
print(f'Number of nodes on internal wall: {num_on_internal_wall}')
print(f'Number of nodes inside boundary: {num_inside_boundary}')
print(f'Number of nodes outside boundary: {num_outside_boundary}')


Number of nodes on external wall: 100
Number of nodes on internal wall: 46
Number of nodes inside boundary: 569
Number of nodes outside boundary: 1547


In [32]:
# セル12: ノードデータの保存

# CSVファイルとして保存
nodes_df.to_csv('../output/nodes_data/nodes_data.csv', index=False)
